Include all the packages needed for this exercise as reported below

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import uproot
import pandas
import scipy
from scipy.optimize import curve_fit

Write an enumeration where are reported the sample id for each sample, considering the same sample_id in AnaTuple.h - https://github.com/hh-italian-group/hh-bbtautau/blob/cmsdas_2019/Analysis/include/AnaTuple.h#L16

In [ ]:
class Samples:
    #fill the enumeration - hint write the Signal_Radion as vector

Define a class for btagging working points

In [ ]:
class btag_wp:
    LOOSE = 0.5426
    MEDIUM = 0.8484
    TIGHT = 0.9535

Here you can find the values of the cross-sections and branching ratios for each mass for the resonance samples (for the non-resonant are already included in the ntuples) and the branching ratio for the two higgses decaying in 2 tau leptons and 2 b-jets

In [1]:
cross_sections = { 250 : 11.52, 260 : 10.31 , 270 : 9.62 , 280 : 8.79 , 300 : 7.20 , 320 : 6.19 , 340 : 5.22 , 
         350 : 4.79 , 400 : 3.37, 450 : 2.20 , 500 : 1.96, 550 : 1.2 , 600 : 1.24, 650 : 0.82 , 
         750 : 0.66, 800 : 0.54, 900 : 0.37}

br_hh = { 250 : 1, 260 : 0.24 , 270 : 0.28 , 280 : 0.31 , 300 : 0.32 , 320 : 0.33 , 340 : 0.32, 
         350 : 0.32, 400 : 0.28, 450 : 0.26, 500 : 0.25, 550 : 0.24, 600 : 0.24, 650 : 0.24, 
         750 : 0.237, 800 : 0.237, 900 : 0.237}

br_hh_bbtt = 2 * 5.809e-01 * 6.256e-02

Open the file, called anaTuples, which are different for each channel. Then create a pandas DataFrame.

In [ ]:
#open the file according where you are working

#path = "/gpfs/ddn/cms/user/cmsdas/2019/hh_bbtautau/anaTuples/" #in Pisa
#path = "/eos/home-m/mgrippo/CMSDAS_2019_hh_bbtautau/anaTuples/" #on Swan

channel = #specify the channel with which you want to work: eTau, muTau or tauTau
signal =  #specify the signal with which you want to work "non-res" or "res"

#open file with uproot and create pandas DataFrame
with uproot.open(path+channel+"_tuple.root") as file:
    tree = file[channel]
    df = tree.arrays('*', outputtype=pandas.DataFrame)

Define QCD contribution, considering that the contribution in the signal region (= opposite-sign for taus and both isolated - os_iso) can be estimated from data subtracting background MC contribution using the following formula:

os_iso = ss_iso * os_antiiso/ss_antiiso

In [ ]:
def CalculateQCD(selection) :
    #specify the region using the branch in anaTuples called region_id
    #Region_Index { Other = 0, OS_Isolated = 1, OS_AntiIsolated = 2, SS_Isolated = 3, SS_AntiIsolated = 4 }
    
    #calculate the number of events for data and bkg MC in each region and with a specific selection passed from outside
    
    #be carefull if the subtracction gives a negative number return 0!
    

Write a method to calculate the significance, taking into account the difference between resonance and non-resonant samples. Remember to calculate the contribution for the background and adding the qcd estimation

In [ ]:
def CalculateSigma(mX, selection, region, is_res) :
    #define how to calculate s/sqrt(s+b)

The aim of the exercise is to calculate the significance for each resonant and non-resonant sample, using different preselection:

* Requiring that we are in the signal region, that means that the two tau leptons should be opposite charge and they should isolated. And requiring that both b-jets pass the medium working point for b-tagging

In [ ]:
b_tag_sel_1 = df.csv_b1 > btag_wp.MEDIUM 
b_tag_sel_2 = df.csv_b2 > btag_wp.MEDIUM 
signal_region = df.region_id == 1

presel = b_tag_sel_1 & b_tag_sel_2 

print('{:10} {:10}'.format('mX', 'sigma'))

if(signal == "non-res") :
    mX = 125
    sigma = CalculateSigma(mX, presel,signal_region,False)
    print('{:<10.2f} {:<10.6f}'.format(mX, sigma))
    
elif(signal == "res") :
    mX_list = np.array(Samples.Signal_Radion)
    mX = [ -x for x in mX_list]
    print(len(mX))
    sigma = np.zeros(len(mX))
    for n in range(len(mX)):
        sigma[n] = CalculateSigma(mX[n], presel,signal_region,True)
        print('{:<10.2f} {:<10.2f}'.format(mX[n], sigma[n]))

    sigma_plt = plt.plot(mX, sigma)
    plt.xlabel(r'$M_X$ [GeV]')
    plt.ylabel(r'Significance')
    plt.title('Significance')
    plt.show()

* As second point it should be added a different mass cut for the resonant candidate, to see how it changes the significance

Define a method to calculate the maximum probable value and the width using the 68 percentile

In [ ]:
def CalcMaxAndWidth(mX):
    #Define a method to calculate the maximum probable value and the width using the 68 percentile

In [ ]:
#repeat the previous calculation but adding this cut on mX
#Pay attention that each calculation of sigma depends on this cut